## Normalized tables

To normalize tables, first I polished the [festival_dataset_clean.csv](https://github.com/Donnie-McGee/Festival-Purchase-Behavior-Analysis/blob/main/Datasets%20%26%20Tables/festival_dataset_clean.csv) to create the [final_festival_dataset](https://github.com/Donnie-McGee/Festival-Purchase-Behavior-Analysis/blob/main/Datasets%20%26%20Tables/final_festival_dataset.csv), which contained no string fields, counting only with integer, boolean and date columns. This is designed to optimize procedural resources.

In order to achieve it, first I selected the fields I was going to work with and then followed 2 different approaches to create normalized dimention tables. I used the same dictionary for both of them, although I added all columns in 2 different steps for an optimal use of it. The approaches consisted in:

1. Using the dictionary and let the code assing the IDs with no specific order.
2. Creating some dimention tables manually, assigning IDs in a specific order (Low=1, Medium=2, High=3, or Individual=1, Couple=2, Group=3).

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\PC\Desktop\Estudio\Analisis de Datos\Proyectos\Festival Purchase Behavior Analysis\Datasets\final_festival_dataset.csv")

#### Approach 1

In [ ]:
# Create a dictionary with the columns that will be used to create the dimension tables 
to_dim_table = [
    "ticket_id",
    "gender",
    "favourite_genre",
    "payment_method"
]

# Create a dictionary to hold the dimension tables
dim_tables = {}
# I will use the column names as keys for the dictionary
for col in to_dim_table:
    # Creates a df with unique values for the column and assigns an ID
    dim_df = df[[col]].drop_duplicates().reset_index(drop=True)
    dim_df[f"{col}_id"] = dim_df.index + 1
    # Creates a new column with IDs starting from 1
    dim_tables[f"dim_{col}"] = dim_df

#### Approach 2

In [ ]:
# Automate all 3 rating columns to avoid repetition
rating_columns = ["satisfaction_level", "cleanliness_rating_level", "security_rating_level"]

rating_levels = ["Low", "Medium", "High"]
rating_ids = [1, 2, 3]

for col in rating_columns:
    dim_tables[f"dim_{col}"] = pd.DataFrame({
        f"{col}": rating_levels,
        f"{col}_id": rating_ids
})

dim_tables["dim_group_type"] = pd.DataFrame({
    "group_type": ["Individual", "Couple", "Group"],
    "group_type_id": [1, 2, 3]
})

dim_tables["dim_age_group"] = pd.DataFrame({
    "age_group": ["18-24", "25-34", "35-44", "45-54", "55+"],
    "age_group_id": [1, 2, 3, 4, 5]
})

dim_tables["dim_ticket_type"] = pd.DataFrame({
    "ticket_type": ["1-day Pass", "3-day Pass", "VIP"],
    "ticket_type_id": [1, 2, 3]
})

# Add the columns of the 2nd approach to the list of columns to be merged back to the main dataframe
to_dim_table.extend([
    "ticket_type",
    "satisfaction_level",
    "cleanliness_rating_level",
    "security_rating_level",
    "group_type",
    "age_group"
])

# Merge the dimension tables back to the main dataframe
# I will sustitute the original columns with the new ID columns
for col in to_dim_table:
    # Merges the dimension tables back to the main dataframe
    # I used a left join to keep all records in the main dataframe
    df = df.merge(dim_tables[f"dim_{col}"], on=col, how='left')
    # Deletes the original column from the main dataframe
    df.drop(col, axis=1, inplace=True)

for i in dim_tables:
    print(dim_tables[i].value_counts())

I found an issue:

- *"ticket_id_id"* column: Due to the sintax in the code, this column name was assigned, so I fixed it with:

In [ ]:
# Rename the column to have a consistent naming convention
df.rename(columns={"ticket_id_id": "ticket_id"}, inplace=True)

#### Saving the dim tables

I decided to keep these discarded tables, along with the way I created them, to document every significant step I took. Initially, they were saved in a different folder, but I later changed it to maintain the original structure I had established.

In [ ]:
# Define the folder path to save the dimension tables
dim_table_folder = r"C:\Users\PC\Desktop\Estudio\Analisis de Datos\Proyectos\Festival Purchase Behavior Analysis\Issues and Fixes\Discarted normalized tables"
# Save each dimension table to a CSV file
for table in dim_tables:
    dim_tables[table].to_csv(f"{dim_table_folder}\{table}.csv", index=False)